## Install the repo

In [ ]:
%rm -rf Visual-Place-Recognition-Project
!git clone -b adaptive_reranking --single-branch --recursive https://github.com/Digre01/Visual-Place-Recognition-Project.git


In [ ]:
%cd Visual-Place-Recognition-Project/image-matching-models
!pip install -e .[all]

In [ ]:
!pip install faiss-cpu
!pip install yacs
!pip install loguru

## Download datasets

In [ ]:
!rm -rf "/kaggle/working/Visual-Place-Recognition-Project/data"

In [60]:
%ls

adaptive_reranking.py
data/
dataset_cosplace_loftr.csv
dataset_cosplace_loftr_validation.csv
dataset_cosplace_superpoint-lg.csv
dataset_cosplace_superpoint-lg_validation.csv
dataset_netvlad_loftr.csv
dataset_netvlad_loftr_validation.csv
dataset_netvlad_superpoint-lg.csv
dataset_netvlad_superpoint-lg_validation.csv
dataset_val.csv
download_datasets.py
image-matching-models/
inliers_full_cosplace_loftr/
inliers_full_cosplace_superpoint-lg/
inliers_full_netvlad_loftr/
inliers_full_netvlad_superpoint-lg/
LICENSE
logistic_comparison.png
logistic_regression.ipynb
logs/
match_queries_preds.py
model_cosplace_loftr.joblib
model_cosplace_loftr_validation.joblib
model_cosplace_superpoint-lg.joblib
model_netvlad_loftr.joblib
model_netvlad_superpoint-lg.joblib
prepare_logistic_data.py
__pycache__/
README.md
reranking.py
start_your_project.ipynb
state.db
trained_models/
train_logReg.py
util.py
VPR-methods-evaluation/
vpr_uncertainty/


In [ ]:
%cd Visual-Place-Recognition-Project/

In [95]:
%%writefile download_datasets.py

URLS = {
    "tokyo_xs": "https://drive.google.com/file/d/15QB3VNKj93027UAQWv7pzFQO1JDCdZj2/view?usp=share_link",
    "sf_xs": "https://drive.google.com/file/d/1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y/view?usp=share_link",
    #"gsv_xs": "https://drive.google.com/file/d/1q7usSe9_5xV5zTfN-1In4DlmF5ReyU_A/view?usp=share_link",
    "svox": "https://drive.google.com/file/d/16iuk8voW65GaywNUQlWAbDt6HZzAJ_t9/view?usp=drive_link"
} 

import os
import gdown
import shutil

os.makedirs("data", exist_ok=True)
for dataset_name, url in URLS.items():
    print(f"Downloading {dataset_name}")
    zip_filepath = f"data/{dataset_name}.zip"
    gdown.download(url, zip_filepath, fuzzy=True)
    shutil.unpack_archive(zip_filepath, extract_dir="data")
    os.remove(zip_filepath)

Overwriting download_datasets.py


In [98]:
%rm -rf /kaggle/working/Visual-Place-Recognition-Project/logs
%rm -rf /kaggle/working/Visual-Place-Recognition-Project/data

In [99]:
!python download_datasets.py

Downloading...
From (original): https://drive.google.com/uc?id=15QB3VNKj93027UAQWv7pzFQO1JDCdZj2
From (redirected): https://drive.google.com/uc?id=15QB3VNKj93027UAQWv7pzFQO1JDCdZj2&confirm=t&uuid=ab784cd4-3aaa-4404-ac0d-78c03b05ea2c
To: /kaggle/working/Visual-Place-Recognition-Project/data/tokyo_xs.zip
100%|████████████████████████████████████████| 141M/141M [00:01<00:00, 86.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y
From (redirected): https://drive.google.com/uc?id=1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y&confirm=t&uuid=e503070a-055a-4832-99c5-df286313ef75
To: /kaggle/working/Visual-Place-Recognition-Project/data/sf_xs.zip
100%|██████████████████████████████████████| 1.03G/1.03G [00:14<00:00, 70.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=16iuk8voW65GaywNUQlWAbDt6HZzAJ_t9
From (redirected): https://drive.google.com/uc?id=16iuk8voW65GaywNUQlWAbDt6HZzAJ_t9&confirm=t&uuid=e92dc69a-d994-479e-b98f-8fcb7e82d361


## Pipeline

In [79]:
%rm dataset_cosplace_loftr.csv
%rm dataset_cosplace_superpoint-lg.csv
%rm dataset_netvlad_loftr.csv
%rm dataset_netvlad_superpoint-lg.csv
%rm -rf inliers_full_cosplace_loftr
%rm -rf inliers_full_cosplace_superpoint-lg
%rm -rf inliers_full_netvlad_loftr
%rm -rf inliers_full_netvlad_superpoint-lg
%rm -rf "/kaggle/working/Visual-Place-Recognition-Project/model_cosplace_loftr.joblib"
%rm -rf "/kaggle/working/Visual-Place-Recognition-Project/model_cosplace_superpoint-lg.joblib"
%rm -rf "/kaggle/working/Visual-Place-Recognition-Project/model_netvlad_loftr.joblib"
%rm -rf "/kaggle/working/Visual-Place-Recognition-Project/model_netvlad_superpoint-lg.joblib"

rm: cannot remove 'dataset_cosplace_loftr.csv': No such file or directory
rm: cannot remove 'dataset_cosplace_superpoint-lg.csv': No such file or directory
rm: cannot remove 'dataset_netvlad_loftr.csv': No such file or directory
rm: cannot remove 'dataset_netvlad_superpoint-lg.csv': No such file or directory


In [ ]:
%ls 
print(gsv_db, gsv_queries)

In [ ]:
import os
import shutil

CONFIGS = [
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "loftr"},
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "superpoint-lg"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "loftr"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "superpoint-lg"}
]



for conf in CONFIGS:
    v_name = conf["vpr"]
    m_name = conf["matcher"]
    log_tag = f"train_{v_name}"
    
    print(f"\n{'='*60}\Running: {v_name.upper()} + {m_name.upper()}\n{'='*60}")
    
    #RETRIVAL
    !python VPR-methods-evaluation/main.py \
        --method={v_name} --backbone={conf['backbone']} --descriptors_dimension={conf['dim']} \
        --num_workers 4 --batch_size 32 \
        --log_dir {log_tag} \
        --image_size 512 512 \
        --database_folder 'data/svox/images/train/gallery' \
        --queries_folder 'data/svox/images/train/queries_sun' \
        --recall_values 1 5 10 20 --num_preds_to_save 20
    

    base_log = f"logs/{log_tag}"
    latest_run = sorted(os.listdir(base_log))[-1]
    preds_dir = os.path.join(base_log, latest_run, "preds")
    print(preds_dir)
    

    #MATCHING
    out_inliers = f"inliers_full_{v_name}_{m_name}"
    !python match_queries_preds.py \
        --preds-dir {preds_dir} \
        --out-dir {out_inliers} \
        --matcher {m_name} \
        --num-preds 20
    
    #TRAINING
    csv_file = f"dataset_{v_name}_{m_name}.csv"
    model_file = f"model_{v_name}_{m_name}.joblib"
    
    !python prepare_logistic_data.py --preds-dir {preds_dir} --inliers-dir {out_inliers} --output-csv {csv_file}
    !python train_logReg.py --csv-path {csv_file} --model-out {model_file}

## Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
import glob

CONFIGS = [
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "loftr"},
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "superpoint-lg"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "loftr"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "superpoint-lg"}
]

plt.figure(figsize=(12, 7))

x_test = np.linspace(0, 100, 500).reshape(-1, 1)

colors = ['blue', 'cyan', 'red', 'orange']
styles = ['-', '--', '-', '--']

for i, conf in enumerate(CONFIGS):
    m_path = f"model_{conf['vpr']}_{conf['matcher']}.joblib"
    if os.path.exists(m_path):
        model = joblib.load(m_path)
        # Calcola la probabilità P(y=1 | x)
        probs = model.predict_proba(x_test)[:, 1]
        
        label = f"{conf['vpr']} + {conf['matcher']}"
        plt.plot(x_test, probs, label=label, color=colors[i], linestyle=styles[i], lw=2)

plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
plt.title("Probability correct match", fontsize=14)
plt.xlabel("Num inliers (Top-1)", fontsize=12)
plt.ylabel("Probability (distance < 25m)", fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.4)
plt.legend(loc='lower right', fontsize=10)
plt.xlim(0, 100) 
plt.ylim(0, 1.05)

plt.savefig("logistic_comparison.png")
plt.show()

## Validation

In [ ]:
import os
import shutil

CONFIGS = [
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "loftr"},
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "superpoint-lg"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "loftr"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "superpoint-lg"}
]



for conf in CONFIGS:
    v_name = conf["vpr"]
    m_name = conf["matcher"]
    log_tag = f"train_{v_name}"
    
    print(f"\n{'='*60} Running: {v_name.upper()} + {m_name.upper()}\n{'='*60}")
    
    !python VPR-methods-evaluation/main.py \
        --method={v_name} --backbone={conf['backbone']} --descriptors_dimension={conf['dim']} \
        --num_workers 4 --batch_size 32 \
        --log_dir {log_tag} \
        --image_size 512 512 \
        --database_folder 'data/sf_xs/val/database' \
        --queries_folder 'data/sf_xs/val/queries' \
        --recall_values 1 5 10 20 --num_preds_to_save 20
    

    base_log = f"logs/{log_tag}"
    latest_run = sorted(os.listdir(base_log))[-1]
    preds_dir = os.path.join(base_log, latest_run, "preds")
    print(preds_dir)
    
    out_inliers = f"inliers_full_{v_name}_{m_name}"
    !python match_queries_preds.py \
        --preds-dir {preds_dir} \
        --out-dir {out_inliers} \
        --matcher {m_name} \
        --num-preds 20
    
    #TRAINING
    csv_file = f"dataset_{v_name}_{m_name}_validation.csv"
    model_file = f"model_{v_name}_{m_name}_validation.joblib"
    
    !python prepare_logistic_data.py --preds-dir {preds_dir} --inliers-dir {out_inliers} --output-csv {csv_file}
    !python train_logReg.py --csv-path {csv_file} --model-out {model_file}

In [55]:
!python train_logReg.py --csv-path "/kaggle/input/cosplace-loftr/dataset_cosplace_loftr.csv" --model-out "model_cosplace_loftr.joblib"

Model saved as model_cosplace_loftr.joblib


In [75]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import precision_score

# Validation parameters
thresholds = np.linspace(0.1, 0.95, 18)
target_precision = 0.99  #Minimum Precision for TOP-1


v_name, m_name = conf["vpr"], conf["matcher"]
model_file =  "model_cosplace_loftr.joblib"

#there is only SF_XS val
val_csv = "/kaggle/input/cosplace-loftr-validation/dataset_cosplace_loftr_validation.csv"

try:
    model = joblib.load(model_file)
    df = pd.read_csv(val_csv)

    
    probs = model.predict_proba(df[['num_inliers']].values)[:, 1]
    
    best_t, max_savings = 0.5, 0.0
    
    for t in thresholds:
        preds = (probs >= t).astype(int)
        precision = precision_score(df['label'], preds, zero_division=0)
        
        if precision >= target_precision:
            #1 match if stopped, 20 matches otherwise
            avg_matches = np.mean([1 if p == 1 else 20 for p in preds])
            savings = (1 - (avg_matches / 20)) * 100
            if savings >= max_savings:
                max_savings = savings
                best_t = t
                
    print(f"{v_name.upper()} + {m_name.upper()} | Selected Threshold: {best_t:.2f} | Validation Savings: {max_savings:.1f}%")
    
except FileNotFoundError:
    print("Error")

COSPLACE + LOFTR | Selected Threshold: 0.65 | Validation Savings: 77.3%


## Testing

In [105]:
CONFIGS = [
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "loftr"},
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "superpoint-lg"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "loftr"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "superpoint-lg"}
]

conf = CONFIGS[0]
log_tag = f"train_{conf['vpr']}"

!python VPR-methods-evaluation/main.py \
        --method={v_name} --backbone={conf['backbone']} --descriptors_dimension={conf['dim']} \
        --num_workers 4 --batch_size 32 \
        --log_dir {log_tag} \
        --image_size 512 512 \
        --database_folder 'data/svox/images/test/gallery' \
        --queries_folder 'data/svox/images/test/queries_night' \
        --recall_values 1 5 10 20 --num_preds_to_save 20

2026-01-24 17:18:04 VPR-methods-evaluation/main.py --method=cosplace --backbone=ResNet18 --descriptors_dimension=512 --num_workers 4 --batch_size 32 --log_dir train_cosplace --image_size 512 512 --database_folder data/svox/images/test/gallery --queries_folder data/svox/images/test/queries_night --recall_values 1 5 10 20 --num_preds_to_save 20
2026-01-24 17:18:04 Arguments: Namespace(positive_dist_threshold=25, method='cosplace', backbone='ResNet18', descriptors_dimension=512, faiss_metric='L2', database_folder='data/svox/images/test/gallery', queries_folder='data/svox/images/test/queries_night', num_workers=4, batch_size=32, log_dir='train_cosplace', device='cuda', recall_values=[1, 5, 10, 20], no_labels=False, num_preds_to_save=20, save_only_wrong_preds=False, image_size=[512, 512], save_descriptors=False, save_for_uncertainty=False, use_labels=True)
2026-01-24 17:18:04 Testing with cosplace with a ResNet18 backbone and descriptors dimension 512
2026-01-24 17:18:04 The outputs are bei

In [107]:
%rm -rf '/kaggle/working/Visual-Place-Recognition-Project/logs/train_netvlad/2026-01-24_14-26-47/preds_loftr'
!python match_queries_preds.py \
--preds-dir 'logs/train_cosplace/2026-01-24_17-18-04/preds' \
--matcher 'loftr' \
--device 'cuda' \
--num-preds 20 \
--logistic-model-path "model_cosplace_loftr.joblib" \
--logistic-prob-threshold 0.65  

/kaggle/working/Visual-Place-Recognition-Project/image-matching-models/matching/third_party/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
100%|█████████████████████████████████████████| 823/823 [21:02<00:00,  1.53s/it]
Total time: 1262.91s
Match: 11805 over 16460


In [108]:
!python adaptive_reranking.py  \
    --preds-dir '/kaggle/working/Visual-Place-Recognition-Project/logs/train_cosplace/2026-01-24_17-18-04/preds' \
    --inliers-dir '/kaggle/working/Visual-Place-Recognition-Project/logs/train_cosplace/2026-01-24_17-18-04/preds_loftr' \
    --num-preds 20 


Eval over 823 query...
100%|████████████████████████████████████████| 823/823 [00:01<00:00, 739.56it/s]

RESULTS RE-RANKING ADAPTIVE
Baseline (Retrieval-only) R@1: 33.29%
Adaptive Matching R@1:       61.12%
Computational cost saving:    28.28%
---------------------------------------------
OTHER DATA:
Recall Adattive:
 R@1: 61.12%
 R@5: 65.37%
 R@10: 66.22%
 R@20: 67.56%
